## 1. 고유한 값, 결측치가 많은 컬럼들을 삭제 후 모델링하면 성능 향상할 것이다

In [113]:
train = pd.read_csv('~/Aiffel/DATAThon/playground-series-s4e11/train.csv')
test = pd.read_csv('~/Aiffel/DATAThon/playground-series-s4e11/test.csv')

In [114]:
train = train.drop(['Name', 'City', 'Profession', 'Sleep Duration', 'Age', 'Degree','id'], axis=1)

In [115]:
# 결측치 처리
train.drop(columns=['CGPA'], inplace=True)
train['Academic_Pressure_missing'] = train['Academic Pressure'].isnull().astype(int)
train['Study_Satisfaction_missing'] = train['Study Satisfaction'].isnull().astype(int)
imputer_ap = SimpleImputer(strategy='mean')
imputer_ss = SimpleImputer(strategy='mean')
train['Academic Pressure'] = imputer_ap.fit_transform(train[['Academic Pressure']])
train['Study Satisfaction'] = imputer_ss.fit_transform(train[['Study Satisfaction']])
train['Work_Pressure_missing'] = train['Work Pressure'].isnull().astype(int)
train['Job_Satisfaction_missing'] = train['Job Satisfaction'].isnull().astype(int)
imputer_wp = SimpleImputer(strategy='mean')
imputer_js = SimpleImputer(strategy='mean')
train['Work Pressure'] = imputer_wp.fit_transform(train[['Work Pressure']])
train['Job Satisfaction'] = imputer_js.fit_transform(train[['Job Satisfaction']])
train = train[train['Financial Stress'].notnull()].copy()

valid_dietary = ['Moderate', 'Unhealthy', 'Healthy']
train['Dietary Habits'] = train['Dietary Habits'].where(train['Dietary Habits'].isin(valid_dietary))
train['Dietary Habits'] = train['Dietary Habits'].fillna(train['Dietary Habits'].mode()[0])

binary_cols = ['Have you ever had suicidal thoughts ?', 'Family History of Mental Illness']
for col in binary_cols:
    train[col] = train[col].map({'Yes': 1, 'No': 0})
train['Gender'] = train['Gender'].map({'Male': 1, 'Female': 0})
train['Working Professional or Student'] = train['Working Professional or Student'].map({'Working Professional': 1, 'Student': 0})

# 수치형 변수 스케일링
scale_cols = ['Work/Study Hours', 'Financial Stress']
# scale_cols = ['Age', 'Sleep Duration', 'Work/Study Hours', 'Financial Stress']
scaler = StandardScaler()
train[scale_cols] = scaler.fit_transform(train[scale_cols])

train = pd.get_dummies(train, columns=['Dietary Habits'], drop_first=True) # 다중공선성 방지를 위해 첫 번째 범주는 제거

In [116]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score
from xgboost import XGBClassifier
import numpy as np

# 결과 저장용 리스트
accuracy_list = []
recall_list = []

# 10회 반복
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(
        train.drop(columns=['Depression']),
        train['Depression'],
        test_size=0.3,
        stratify=train['Depression'],
        random_state=i  # 반복마다 seed 변경
    )

    model = XGBClassifier(
        n_estimators=100,
        max_depth=4,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        use_label_encoder=False,
        eval_metric='logloss',
        n_jobs=-1
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)

    accuracy_list.append(acc)
    recall_list.append(rec)

# 결과 출력
print("Accuracy (10회):", np.round(accuracy_list, 4))
print("Recall (10회):   ", np.round(recall_list, 4))
print(f"\n평균 Accuracy: {np.mean(accuracy_list):.4f}")
print(f"평균 Recall:   {np.mean(recall_list):.4f}")


/Users/choieunseo/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [20:40:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/choieunseo/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [20:40:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/choieunseo/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [20:40:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/choieunseo/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [20:40:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  w

Accuracy (10회): [0.913  0.9132 0.9135 0.9134 0.9134 0.9126 0.914  0.9118 0.9131 0.9118]
Recall (10회):    [0.6815 0.6733 0.6793 0.6747 0.679  0.6728 0.6823 0.6711 0.6777 0.6731]

평균 Accuracy: 0.9130
평균 Recall:   0.6765


## 2. 학생 / 직장인 데이터를 분리하여 모델링하면 성능 향상할 것이다

In [138]:
train = pd.read_csv('~/Aiffel/DATAThon/playground-series-s4e11/train.csv')
test = pd.read_csv('~/Aiffel/DATAThon/playground-series-s4e11/test.csv')

In [139]:
train.drop(columns=['id'], inplace=True)
train.drop(columns=['Name'], inplace=True)

# 이상치, 알 수 없는 값들 제거
delete_values = [
    'Indore', 'Pune', 'Moderate', 'Unhealthy', 'Sleep_Duration',
    'Work_Study_Hours', 'No', '45', '49 hours', '55-66 hours', '40-45 hours', 
    '9-5 hours', '10-6 hours', '9-6 hours', '9-5', '45-48 hours', '35-36 hours'
]

train = train[~train['Sleep Duration'].isin(delete_values)].copy()

def convert_sleep_to_hours(val):
    try:
        val = str(val).strip().lower()

        # 특별 처리: 'than n hours' → 'less than n hours' 간주
        if 'than' in val and 'less' not in val and 'more' not in val:
            match = re.search(r'\d+', val)
            if match:
                return float(match.group()) - 0.5

        # Less than n hours → n - 0.5
        if 'less than' in val:
            match = re.search(r'\d+', val)
            if match:
                return float(match.group()) - 0.5

        # More than n hours → n + 0.5
        elif 'more than' in val:
            match = re.search(r'\d+', val)
            if match:
                return float(match.group()) + 0.5

        # 정확히 n hours → 숫자만 추출
        elif re.match(r'^\d+\s*hours$', val):
            return float(re.findall(r'\d+', val)[0])

        # n-m hours 또는 n–n → 평균값
        elif re.search(r'\d+\s*[-–~]\s*\d+', val):
            nums = [int(n) for n in re.findall(r'\d+', val)]
            if len(nums) == 2:
                return sum(nums) / 2

        # 숫자만 → 그대로
        elif re.match(r'^\d+(\.\d+)?$', val):
            return float(val)

        # 나머지는 이상값으로 간주
        else:
            return np.nan

    except:
        return np.nan

train['Sleep Duration'] = train['Sleep Duration'].apply(convert_sleep_to_hours)

work_train = train[train['Working Professional or Student'] == 'Working Professional']
stu_train = train[train['Working Professional or Student'] == 'Student']

#직장인 테이블 결측치 제거
work_cols_to_drop = ['Academic Pressure', 'Study Satisfaction', 'CGPA']
work_train = work_train.drop(columns=work_cols_to_drop)

#학생 테이블 결측치 제거
stu_cols_to_drop = ['Work Pressure', 'Job Satisfaction', 'Profession']
stu_train = stu_train.drop(columns=stu_cols_to_drop)

# 1. CGPA 컬럼 제거
stu_train.drop(columns=['CGPA'], inplace=True)

# 2. Academic Pressure 결측 여부 파생 변수
stu_train['Academic_Pressure_missing'] = stu_train['Academic Pressure'].isnull().astype(int)

# 3. Study Satisfaction 결측 여부 파생 변수
stu_train['Study_Satisfaction_missing'] = stu_train['Study Satisfaction'].isnull().astype(int)

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')  # 또는 'median', 'most_frequent'
stu_train['Academic Pressure'] = imputer.fit_transform(stu_train[['Academic Pressure']])
stu_train['Study Satisfaction'] = imputer.fit_transform(stu_train[['Study Satisfaction']])

# 4. Work_Pressure_missing 결측 여부 파생 변수
work_train['Work_Pressure_missing'] = work_train['Work Pressure'].isnull().astype(int)
# 5. Job_Satisfaction_missing 결측 여부 파생 변수
work_train['Job_Satisfaction_missing'] = work_train['Job Satisfaction'].isnull().astype(int)

work_train['Work Pressure'] = imputer.fit_transform(work_train[['Work Pressure']])
work_train['Job Satisfaction'] = imputer.fit_transform(work_train[['Job Satisfaction']])

work_train = work_train[work_train['Financial Stress'].notnull()].copy()
stu_train = stu_train[stu_train['Financial Stress'].notnull()].copy()

work_train['Profession'].fillna('Missing', inplace=True)

from sklearn.preprocessing import StandardScaler

scale_cols = ['Age', 'Sleep Duration', 'Work/Study Hours', 'Financial Stress']

scaler = StandardScaler()
work_train[scale_cols] = scaler.fit_transform(work_train[scale_cols])
stu_train[scale_cols] = scaler.fit_transform(stu_train[scale_cols])

degree_group_map = {
    # 의학/보건
    'MD': 'Medical',
    'MBBS': 'Medical',
    'B.Pharm': 'Pharmacy',
    'M.Pharm': 'Pharmacy',
    'MPharm': 'Pharmacy',
    'P.Pharm': 'Pharmacy',
    'S.Pharm': 'Pharmacy',
    'N.Pharm': 'Pharmacy',

    # 공학/기술
    'B.Tech': 'Engineering',
    'M.Tech': 'Engineering',
    'ME': 'Engineering',
    'MTech': 'Engineering',
    'M_Tech': 'Engineering',
    'BE': 'Engineering',
    'BCA': 'Engineering',
    'MCA': 'Engineering',
    'E.Tech': 'Engineering',
    'S.Tech': 'Engineering',
    'LLTech': 'Engineering',
    'LLCom': 'Engineering',

    # 인문/사회/비즈니스
    'BBA': 'Business',
    'MBA': 'Business',
    'M. Business Analyst': 'Business',
    'B.Com': 'Commerce',
    'M.Com': 'Commerce',
    'P.Com': 'Commerce',
    'LLB': 'Law',
    'LLM': 'Law',
    'LLBA': 'Law',
    'LL.Com': 'Law',
    'LL B.Ed': 'Education',
    'B.Ed': 'Education',
    'M.Ed': 'Education',
    'L.Ed': 'Education',
    'K.Ed': 'Education',
    'LLEd': 'Education',
    'BEd': 'Education',

    # 과학
    'BSc': 'Science',
    'MSc': 'Science',
    'B.Sc': 'Science',

    # 기타, 건축, 호텔경영 등
    'BHM': 'Hospitality',
    'MHM': 'Hospitality',
    'B.Arch': 'Architecture',
    'M.Arch': 'Architecture',
    'BArch': 'Architecture',
    'B.B.Arch': 'Architecture',

    # 학위 및 학교 졸업
    'PhD': 'PhD',
    'Class 12': 'School',
    'Class 11': 'School',
}

work_train['degree_group'] = work_train['Degree'].apply(lambda x: degree_group_map.get(x, 'Other'))
stu_train['degree_group'] = stu_train['Degree'].apply(lambda x: degree_group_map.get(x, 'Other'))

work_train=work_train.drop('Degree',axis=1)
stu_train=stu_train.drop('Degree',axis=1)

# 정상 값만 남기고 나머지는 NaN으로
valid_dietary = ['Moderate', 'Unhealthy', 'Healthy']
work_train['Dietary Habits'] = work_train['Dietary Habits'].where(work_train['Dietary Habits'].isin(valid_dietary))
stu_train['Dietary Habits'] = stu_train['Dietary Habits'].where(stu_train['Dietary Habits'].isin(valid_dietary))

# 결측값은 최빈값으로 ㅁ
work_train['Dietary Habits'] = work_train['Dietary Habits'].fillna(work_train['Dietary Habits'].mode()[0])
stu_train['Dietary Habits'] = stu_train['Dietary Habits'].fillna(stu_train['Dietary Habits'].mode()[0])



/var/folders/hf/5rb3dt8x3mq707hsrnthhx2w0000gn/T/ipykernel_1363/966698980.py:95: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  work_train['Profession'].fillna('Missing', inplace=True)


In [140]:
from sklearn.preprocessing import LabelEncoder

# Gender, Working Status - Binary
list = [work_train, stu_train]

for train in list :
    train['Gender'] = train['Gender'].map({'Male': 1, 'Female': 0})
    train['Working Professional or Student'] = train['Working Professional or Student'].map({'Working Professional': 1, 'Student': 0})

# Degree, Dietary Habits - One-hot encoding
work_train = pd.get_dummies(work_train, columns=['degree_group', 'Dietary Habits'], drop_first=True)
stu_train = pd.get_dummies(stu_train, columns=['degree_group', 'Dietary Habits'], drop_first=True)

# Profession, City - Target encoding (또는 frequency encoding)
# 예: Target Mean Encoding (Depression 컬럼 사용)
for col in ['Profession', 'City']:
    target_mean = work_train.groupby(col)['Depression'].mean()
    work_train[col + '_target'] = work_train[col].map(target_mean)

# 이후 필요시 원본 컬럼 제거
work_train.drop(columns=['Profession', 'City'], inplace=True)

## 2 ## 
for col in ['City']:
    target_mean = stu_train.groupby(col)['Depression'].mean()
    stu_train[col + '_target'] = stu_train[col].map(target_mean)

# 이후 필요시 원본 컬럼 제거
stu_train.drop(columns=[ 'City'], inplace=True)

binary_cols = ['Have you ever had suicidal thoughts ?', 'Family History of Mental Illness']
for col in binary_cols:
    work_train[col] = work_train[col].map({'Yes': 1, 'No': 0})
    stu_train[col] = stu_train[col].map({'Yes': 1, 'No': 0})

In [141]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score
import numpy as np

# 📌 평가 지표 저장 리스트 초기화
work_acc_list, work_rec_list = [], []
stu_acc_list, stu_rec_list = [], []
all_acc_list, all_rec_list = [], []

# 📌 공통 모델 하이퍼파라미터 (random_state는 루프 내에서 넣음)
base_params = {
    'n_estimators': 100,
    'max_depth': 4,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'use_label_encoder': False,
    'eval_metric': 'logloss',
    'n_jobs': -1
}

# 📌 학습/테스트 분할 함수 정의 (seed 사용)
def get_train_test_dataset(df, seed):
    y = df['Depression']
    X = df.drop('Depression', axis=1)
    return train_test_split(X, y, test_size=0.3, stratify=y, random_state=seed)

# 📌 10회 반복 학습 및 평가
for seed in range(10):
    # 데이터 분할 (seed 적용)
    work_X_train, work_X_test, work_y_train, work_y_test = get_train_test_dataset(work_train, seed)
    stu_X_train, stu_X_test, stu_y_train, stu_y_test = get_train_test_dataset(stu_train, seed)

    # 직장인 모델 (seed 적용)
    work_model = XGBClassifier(**base_params, random_state=seed)
    work_model.fit(work_X_train, work_y_train)
    work_y_pred = work_model.predict(work_X_test)
    work_acc_list.append(accuracy_score(work_y_test, work_y_pred))
    work_rec_list.append(recall_score(work_y_test, work_y_pred))

    # 학생 모델 (seed 적용)
    stu_model = XGBClassifier(**base_params, random_state=seed)
    stu_model.fit(stu_X_train, stu_y_train)
    stu_y_pred = stu_model.predict(stu_X_test)
    stu_acc_list.append(accuracy_score(stu_y_test, stu_y_pred))
    stu_rec_list.append(recall_score(stu_y_test, stu_y_pred))

    # 전체 데이터 통합 평가
    y_true_all = np.concatenate([work_y_test, stu_y_test])
    y_pred_all = np.concatenate([work_y_pred, stu_y_pred])
    all_acc_list.append(accuracy_score(y_true_all, y_pred_all))
    all_rec_list.append(recall_score(y_true_all, y_pred_all))

# 📌 결과 출력 함수
def print_results(title, acc_list, rec_list):
    print(f"📊 [{title}] Accuracy (10회):", np.round(acc_list, 4))
    print(f"📊 [{title}] Recall    (10회):", np.round(rec_list, 4))
    print(f"▶ 평균 Accuracy: {np.mean(acc_list):.4f}")
    print(f"▶ 평균 Recall:   {np.mean(rec_list):.4f}")
    print()

# 📌 결과 출력
print_results("직장인", work_acc_list, work_rec_list)
print_results("학생", stu_acc_list, stu_rec_list)
print_results("전체", all_acc_list, all_rec_list)


/Users/choieunseo/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [20:46:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/choieunseo/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [20:46:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/choieunseo/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [20:46:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/choieunseo/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [20:46:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  w

📊 [직장인] Accuracy (10회): [0.9605 0.9623 0.9618 0.9611 0.9605 0.9607 0.9613 0.9635 0.9615 0.9621]
📊 [직장인] Recall    (10회): [0.6793 0.6876 0.68   0.6717 0.6779 0.6728 0.6746 0.7017 0.6829 0.688 ]
▶ 평균 Accuracy: 0.9615
▶ 평균 Recall:   0.6817

📊 [학생] Accuracy (10회): [0.8505 0.8454 0.8511 0.846  0.8439 0.8447 0.8506 0.8462 0.846  0.8533]
📊 [학생] Recall    (10회): [0.8918 0.8861 0.8916 0.8808 0.8818 0.8827 0.888  0.8857 0.8818 0.8941]
▶ 평균 Accuracy: 0.8478
▶ 평균 Recall:   0.8864

📊 [전체] Accuracy (10회): [0.9387 0.9391 0.9398 0.9383 0.9374 0.9377 0.9394 0.9402 0.9386 0.9405]
📊 [전체] Recall    (10회): [0.8151 0.8144 0.8152 0.8053 0.8082 0.8069 0.8109 0.8193 0.81   0.8197]
▶ 평균 Accuracy: 0.9390
▶ 평균 Recall:   0.8125



## 3. 결측치 임퓨팅 / 파생 변수 만든 후 모델링하면 성능 향상할 것이다

In [134]:
train = pd.read_csv('~/Aiffel/DATAThon/playground-series-s4e11/train.csv')
test = pd.read_csv('~/Aiffel/DATAThon/playground-series-s4e11/test.csv')

In [135]:
# 필요없는 컬럼 삭제
train.drop(columns=['id'], inplace=True)
train.drop(columns=['Name'], inplace=True)

# 결측치 처리
train.drop(columns=['CGPA'], inplace=True)
train['Academic_Pressure_missing'] = train['Academic Pressure'].isnull().astype(int)
train['Study_Satisfaction_missing'] = train['Study Satisfaction'].isnull().astype(int)
imputer = SimpleImputer(strategy='mean')
train['Academic Pressure'] = imputer.fit_transform(train[['Academic Pressure']])
train['Study Satisfaction'] = imputer.fit_transform(train[['Study Satisfaction']])
train['Work_Pressure_missing'] = train['Work Pressure'].isnull().astype(int)
train['Job_Satisfaction_missing'] = train['Job Satisfaction'].isnull().astype(int)
train['Work Pressure'] = imputer.fit_transform(train[['Work Pressure']])
train['Job Satisfaction'] = imputer.fit_transform(train[['Job Satisfaction']])
train = train[train['Financial Stress'].notnull()].copy()

# 범주형 변수 처리
delete_values = [
    'Indore', 'Pune', 'Moderate', 'Unhealthy', 'Sleep_Duration',
    'Work_Study_Hours', 'No', '45', '49 hours', '55-66 hours', '40-45 hours', 
    '9-5 hours', '10-6 hours', '9-6 hours', '9-5', '45-48 hours', '35-36 hours'
]
train = train[~train['Sleep Duration'].isin(delete_values)].copy()
def convert_sleep_to_hours(val):
    try:
        val = str(val).strip().lower()
        if 'than' in val and 'less' not in val and 'more' not in val:
            match = re.search(r'\d+', val)
            if match:
                return float(match.group()) - 0.5
        if 'less than' in val:
            match = re.search(r'\d+', val)
            if match:
                return float(match.group()) - 0.5
        elif 'more than' in val:
            match = re.search(r'\d+', val)
            if match:
                return float(match.group()) + 0.5
        elif re.match(r'^\d+\s*hours$', val):
            return float(re.findall(r'\d+', val)[0])
        elif re.search(r'\d+\s*[-–~]\s*\d+', val):
            nums = [int(n) for n in re.findall(r'\d+', val)]
            if len(nums) == 2:
                return sum(nums) / 2
        elif re.match(r'^\d+(\.\d+)?$', val):
            return float(val)
        else:
            return np.nan
    except:
        return np.nan
train['Sleep Duration'] = train['Sleep Duration'].apply(convert_sleep_to_hours)
degree_group_map = {
    'MD': 'Medical',
    'MBBS': 'Medical',
    'B.Pharm': 'Pharmacy',
    'M.Pharm': 'Pharmacy',
    'MPharm': 'Pharmacy',
    'P.Pharm': 'Pharmacy',
    'S.Pharm': 'Pharmacy',
    'N.Pharm': 'Pharmacy',
    'B.Tech': 'Engineering',
    'M.Tech': 'Engineering',
    'ME': 'Engineering',
    'MTech': 'Engineering',
    'M_Tech': 'Engineering',
    'BE': 'Engineering',
    'BCA': 'Engineering',
    'MCA': 'Engineering',
    'E.Tech': 'Engineering',
    'S.Tech': 'Engineering',
    'LLTech': 'Engineering',
    'LLCom': 'Engineering',
    'BBA': 'Business',
    'MBA': 'Business',
    'M. Business Analyst': 'Business',
    'B.Com': 'Commerce',
    'M.Com': 'Commerce',
    'P.Com': 'Commerce',
    'LLB': 'Law',
    'LLM': 'Law',
    'LLBA': 'Law',
    'LL.Com': 'Law',
    'LL B.Ed': 'Education',
    'B.Ed': 'Education',
    'M.Ed': 'Education',
    'L.Ed': 'Education',
    'K.Ed': 'Education',
    'LLEd': 'Education',
    'BEd': 'Education',
    'BSc': 'Science',
    'MSc': 'Science',
    'B.Sc': 'Science',
    'BHM': 'Hospitality',
    'MHM': 'Hospitality',
    'B.Arch': 'Architecture',
    'M.Arch': 'Architecture',
    'BArch': 'Architecture',
    'B.B.Arch': 'Architecture',
    'PhD': 'PhD',
    'Class 12': 'School',
    'Class 11': 'School',
}
train['degree_group'] =train['Degree'].apply(lambda x: degree_group_map.get(x, 'Other'))
train=train.drop('Degree',axis=1)

top_cities = train['City'].value_counts().nlargest(15).index
train['City'] = train['City'].where(train['City'].isin(top_cities), other='Other')

valid_dietary = ['Moderate', 'Unhealthy', 'Healthy']
train['Dietary Habits'] = train['Dietary Habits'].where(train['Dietary Habits'].isin(valid_dietary))
train['Dietary Habits'] = train['Dietary Habits'].fillna(train['Dietary Habits'].mode()[0])

binary_cols = ['Have you ever had suicidal thoughts ?', 'Family History of Mental Illness']
for col in binary_cols:
    train[col] = train[col].map({'Yes': 1, 'No': 0})
train['Gender'] = train['Gender'].map({'Male': 1, 'Female': 0})
train['Working Professional or Student'] = train['Working Professional or Student'].map({'Working Professional': 1, 'Student': 0})

# 수치형 변수 스케일링
scale_cols = ['Sleep Duration', 'Work/Study Hours', 'Financial Stress']
# scale_cols = ['Age', 'Sleep Duration', 'Work/Study Hours', 'Financial Stress']
scaler = StandardScaler()
train[scale_cols] = scaler.fit_transform(train[scale_cols])

# 원핫인코딩 + 타겟인코딩
train = pd.get_dummies(train, columns=['degree_group', 'Dietary Habits'], drop_first=True) # 다중공선성 방지를 위해 첫 번째 범주는 제거
for col in ['Profession', 'City']:
    target_mean = train.groupby(col)['Depression'].mean()
    train[col + '_target'] = train[col].map(target_mean)
    

train.drop(columns=['Profession', 'City'], inplace=True)

In [136]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score
from xgboost import XGBClassifier
import numpy as np

# 결과 저장용 리스트
accuracy_list = []
recall_list = []

# 10회 반복
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(
        train.drop(columns=['Depression']),
        train['Depression'],
        test_size=0.3,
        stratify=train['Depression'],
        random_state=i  # 반복마다 seed 변경
    )

    model = XGBClassifier(
        n_estimators=100,
        max_depth=4,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        use_label_encoder=False,
        eval_metric='logloss',
        n_jobs=-1
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)

    accuracy_list.append(acc)
    recall_list.append(rec)

# 결과 출력
print("Accuracy (10회):", np.round(accuracy_list, 4))
print("Recall (10회):   ", np.round(recall_list, 4))
print(f"\n평균 Accuracy: {np.mean(accuracy_list):.4f}")
print(f"평균 Recall:   {np.mean(recall_list):.4f}")

/Users/choieunseo/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [20:45:46] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/choieunseo/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [20:45:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/choieunseo/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [20:45:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/choieunseo/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [20:45:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  w

Accuracy (10회): [0.938  0.9386 0.9383 0.9364 0.9391 0.9372 0.9384 0.9389 0.94   0.9377]
Recall (10회):    [0.8061 0.8092 0.8114 0.8038 0.8074 0.8086 0.8065 0.8156 0.8191 0.8109]

평균 Accuracy: 0.9383
평균 Recall:   0.8099


## 4. ANOVA

| **가설**                           | **Accuracy (평균)** | **Recall (평균)** | **Accuracy (10회 측정값)**                                                            | **Recall (10회 측정값)**                                                              |
| -------------------------------- | ----------------- | --------------- | --------------------------------------------------------------------------------- | --------------------------------------------------------------------------------- |
| **가설 1**<br>(고유값 및 결측치 많은 컬럼 삭제) | 0.9130            | 0.6765          | \[0.913, 0.9132, 0.9135, 0.9134, 0.9134, 0.9126, 0.914, 0.9118, 0.9131, 0.9118]   | \[0.6815, 0.6733, 0.6793, 0.6747, 0.679, 0.6728, 0.6823, 0.6711, 0.6777, 0.6731]  |
| **가설 2**<br>(학생/직장인 분리 모델링)      | 0.9390        | 0.8125     | \[0.9387, 0.9391, 0.9398, 0.9383, 0.9374, 0.9377, 0.9394, 0.9402, 0.9386, 0.9405] | \[0.8151, 0.8144, 0.8152, 0.8053, 0.8082, 0.8069, 0.8109, 0.8193, 0.81, 0.8197]   |
| **가설 3**<br>(결측치 임퓨팅 + 파생변수 생성)  | 0.9383            | 0.8099          | \[0.938, 0.9386, 0.9383, 0.9364, 0.9391, 0.9372, 0.9384, 0.9389, 0.94, 0.9377]    | \[0.8061, 0.8092, 0.8114, 0.8038, 0.8074, 0.8086, 0.8065, 0.8156, 0.8191, 0.8109] |

In [2]:
import scipy.stats as stats
import pandas as pd

# Accuracy 데이터
acc_h1 = [0.913, 0.9132, 0.9135, 0.9134, 0.9134, 0.9126, 0.914, 0.9118, 0.9131, 0.9118]
acc_h2 = [0.9387, 0.9391, 0.9398, 0.9383, 0.9374, 0.9377, 0.9394, 0.9402, 0.9386, 0.9405]
acc_h3 = [0.938, 0.9386, 0.9383, 0.9364, 0.9391, 0.9372, 0.9384, 0.9389, 0.94, 0.9377]

# Recall 데이터
rec_h1 = [0.6815, 0.6733, 0.6793, 0.6747, 0.679, 0.6728, 0.6823, 0.6711, 0.6777, 0.6731]
rec_h2 = [0.8151, 0.8144, 0.8152, 0.8053, 0.8082, 0.8069, 0.8109, 0.8193, 0.81, 0.8197]
rec_h3 = [0.8061, 0.8092, 0.8114, 0.8038, 0.8074, 0.8086, 0.8065, 0.8156, 0.8191, 0.8109]


In [3]:
from scipy.stats import levene

# Accuracy에 대한 등분산성 검정
levene_acc = levene(acc_h1, acc_h2, acc_h3)
print("Levene’s test for Accuracy:")
print(f"Statistic = {levene_acc.statistic:.4f}, p-value = {levene_acc.pvalue:.4f}")

# Recall에 대한 등분산성 검정
levene_rec = levene(rec_h1, rec_h2, rec_h3)
print("\nLevene’s test for Recall:")
print(f"Statistic = {levene_rec.statistic:.4f}, p-value = {levene_rec.pvalue:.4f}")


Levene’s test for Accuracy:
Statistic = 0.7865, p-value = 0.4656

Levene’s test for Recall:
Statistic = 0.3843, p-value = 0.6846


Levene’s(리빈스) 검정의 귀무가설(H₀):

세 그룹의 분산이 서로 같다 (등분산이다)

p-value가 0.05보다 크므로, 귀무가설을 기각할 수 없음

즉, 세 집단의 분산이 통계적으로 유의미하게 다르지 않다 → 등분산성이 있다는 해석이 가능함

In [144]:
print(stats.shapiro(acc_h1))
print(stats.shapiro(acc_h2))
print(stats.shapiro(acc_h3))

print(stats.shapiro(rec_h1))
print(stats.shapiro(rec_h2))
print(stats.shapiro(rec_h3))

ShapiroResult(statistic=np.float64(0.8991025445855032), pvalue=np.float64(0.21417490269488815))
ShapiroResult(statistic=np.float64(0.9726550536536342), pvalue=np.float64(0.9143110414963821))
ShapiroResult(statistic=np.float64(0.990107053988515), pvalue=np.float64(0.9969422624001061))
ShapiroResult(statistic=np.float64(0.9207633142090637), pvalue=np.float64(0.36336527152231446))
ShapiroResult(statistic=np.float64(0.9431801584219465), pvalue=np.float64(0.5889112848621444))
ShapiroResult(statistic=np.float64(0.9361914536774865), pvalue=np.float64(0.5114632857063324))


🔍 Shapiro-Wilk 정규성 검정 요약
Shapiro-Wilk 검정은 주어진 데이터가 정규 분포를 따르는지를 확인하는 검정입니다.

귀무가설(H₀): 데이터는 정규분포를 따른다.

대립가설(H₁): 데이터는 정규분포를 따르지 않는다.

p-value > 0.05 → 정규성 만족 (H₀ 채택)

p-value ≤ 0.05 → 정규성 위반 (H₀ 기각)

| 통계량 (`statistic`) | p-value | 정규성 만족 여부       |
| ----------------- | ------- | --------------- |
| 0.8991            | 0.2142  | ✅ 만족 (p > 0.05) |
| 0.9727            | 0.9143  | ✅ 만족            |
| 0.9901            | 0.9969  | ✅ 만족            |
| 0.9208            | 0.3634  | ✅ 만족            |
| 0.9432            | 0.5889  | ✅ 만족            |
| 0.9362            | 0.5115  | ✅ 만족            |

모든 데이터셋에서 p-value가 모두 0.05보다 큽니다.
따라서 모든 케이스에서 정규성 가정을 만족한다고 볼 수 있습니다.

이 결과는 ANOVA나 Tukey HSD처럼 정규성 가정을 전제로 하는 통계검정을 수행하는 데에 적합합니다.

In [146]:
f_acc, p_acc = stats.f_oneway(acc_h1, acc_h2, acc_h3)

# 🎯 Recall에 대한 분산분석
f_rec, p_rec = stats.f_oneway(rec_h1, rec_h2, rec_h3)

print(f"[Accuracy] F = {f_acc:.4f}, p = {p_acc:.4f}")
print(f"[Recall]   F = {f_rec:.4f}, p = {p_rec:.4f}")

[Accuracy] F = 2531.0216, p = 0.0000
[Recall]   F = 2915.2928, p = 0.0000


✅ 1. 정규성 만족
앞서 Shapiro-Wilk 검정 결과에서 모든 그룹의 정규성 만족 → OK
✔️ ANOVA 수행 가능 조건 충족

✅ 2. 분산분석 (ANOVA) 결과
```
[Accuracy] F = 2531.0216, p = 0.0000
[Recall]   F = 2915.2928, p = 0.0000
```
🔍 해석:
귀무가설(H₀): 세 그룹의 평균은 모두 같다

대립가설(H₁): 적어도 하나의 그룹은 평균이 다르다

✅ p-value ≪ 0.05 → 귀무가설 기각
👉 즉, 세 그룹 중 적어도 하나는 평균이 유의하게 다르다!
따라서:

✅ 3. 사후검정 (Tukey HSD) 수행 필요
분산분석은 “차이가 있다”는 것까지만 알려줍니다.
어느 그룹 간에 차이가 있는지는 알려주지 않기 때문에,
→ 바로 이어서 Tukey HSD 사후검정을 해야 합니다

In [127]:
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Accuracy 예시
acc_1 = [0.913, 0.9132, 0.9135, 0.9134, 0.9134, 0.9126, 0.914, 0.9118, 0.9131, 0.9118]
acc_2 = [0.9387, 0.9391, 0.9398, 0.9383, 0.9374, 0.9377, 0.9394, 0.9402, 0.9386, 0.9405]
acc_3 = [0.938, 0.9386, 0.9383, 0.9364, 0.9391, 0.9372, 0.9384, 0.9389, 0.94, 0.9377]

# 그룹 이름 지정
acc = acc_1 + acc_2 + acc_3
group = ['hypothesis_1']*10 + ['hypothesis_2']*10 + ['hypothesis_3']*10


# DataFrame 생성
df = pd.DataFrame({'accuracy': acc, 'group': group})

# Tukey’s HSD test 수행
tukey_result = pairwise_tukeyhsd(endog=df['accuracy'], groups=df['group'], alpha=0.05)
print(tukey_result)


      Multiple Comparison of Means - Tukey HSD, FWER=0.05      
   group1       group2    meandiff p-adj   lower  upper  reject
---------------------------------------------------------------
hypothesis_1 hypothesis_2    0.026    0.0   0.025  0.027   True
hypothesis_1 hypothesis_3   0.0253    0.0  0.0242 0.0263   True
hypothesis_2 hypothesis_3  -0.0007 0.2213 -0.0017 0.0003  False
---------------------------------------------------------------


| 그룹명           | 평균 AUC  |
| ------------- | ------- |
| hypothesis\_1 | 약 0.913 |
| hypothesis\_2 | 약 0.939 |
| hypothesis\_3 | 약 0.938 |

hypothesis_1은 다른 두 그룹보다 AUC가 유의하게 낮음

hypothesis_2와 hypothesis_3 간에는 차이 없음 → 성능 유사



In [147]:
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Recall 값 예시
rec_1 = [0.6815, 0.6733, 0.6793, 0.6747, 0.6790, 0.6728, 0.6823, 0.6711, 0.6777, 0.6731]
rec_2 = [0.8151, 0.8144, 0.8152, 0.8053, 0.8082, 0.8069, 0.8109, 0.8193, 0.81, 0.8197]
rec_3 = [0.8061, 0.8092, 0.8114, 0.8038, 0.8074, 0.8086, 0.8065, 0.8156, 0.8191, 0.8109]

# 그룹 이름 지정
recall = rec_1 + rec_2 + rec_3
group = ['hypothesis_1'] * 10 + ['hypothesis_2'] * 10 + ['hypothesis_3'] * 10

# DataFrame 생성
df = pd.DataFrame({'recall': recall, 'group': group})

# Tukey’s HSD test 수행
tukey_result = pairwise_tukeyhsd(endog=df['recall'], groups=df['group'], alpha=0.05)

print(tukey_result)


      Multiple Comparison of Means - Tukey HSD, FWER=0.05      
   group1       group2    meandiff p-adj   lower  upper  reject
---------------------------------------------------------------
hypothesis_1 hypothesis_2    0.136    0.0   0.131 0.1411   True
hypothesis_1 hypothesis_3   0.1334    0.0  0.1283 0.1384   True
hypothesis_2 hypothesis_3  -0.0026 0.4097 -0.0077 0.0024  False
---------------------------------------------------------------


🔍 ✅ 비교: hypothesis_2 vs hypothesis_3

📌 귀무가설 (Null Hypothesis, H₀)
두 그룹의 평균은 같다
→ 𝜇₂ = 𝜇₃
→ recall의 평균 차이 = 0

📌 대립가설 (Alternative Hypothesis, H₁)
두 그룹의 평균은 다르다
→ 𝜇₂ ≠ 𝜇₃
→ recall의 평균 차이 ≠ 0

🔬 해석 결과
p-value = 0.4097 > 0.05 → 귀무가설 기각하지 못함

즉, 통계적으로 평균에 차이가 있다고 보기 어렵다

✅ 결론 정리
가설(H₀): “hypothesis_2와 hypothesis_3는 같은 평균 recall을 가진다.”
결과: “데이터 상에서는 차이가 나지만, 이 차이가 우연에 의한 것일 수도 있다”
→ 따라서 H₀를 유지함 → "차이 없음"으로 판단

❌ hypothesis_2 vs hypothesis_3
평균 차이: -0.0026

p-value: 0.4097

신뢰구간: (-0.0077, 0.0024)

reject = False → 차이 없음

👉 결론: hypothesis_2와 hypothesis_3 간 recall 차이는 통계적으로 유의하지 않음

hypothesis_2와 hypothesis_3 간에는 유의미한 차이 없음 → 성능은 유사함

In [129]:
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Recall 데이터
recall_1 = [0.6815, 0.6733, 0.6793, 0.6747, 0.679, 0.6728, 0.6823, 0.6711, 0.6777, 0.6731]  # 낮음
recall_2 = [0.8151, 0.8144, 0.8152, 0.8053, 0.8082, 0.8069, 0.8109, 0.8193, 0.81, 0.8197]  # 높음
recall_3 = [0.8061, 0.8092, 0.8114, 0.8038, 0.8074, 0.8086, 0.8065, 0.8156, 0.8191, 0.8109]  # 중간

# 평균 높은 순으로 group 라벨 매핑
recall = recall_1 + recall_2 + recall_3
group = ['hypothesis_1']*10 + ['hypothesis_2']*10 + ['hypothesis_3']*10

# DataFrame 생성
df = pd.DataFrame({'recall': recall, 'group': group})

# Tukey HSD 수행
tukey_result = pairwise_tukeyhsd(endog=df['recall'], groups=df['group'], alpha=0.05)
print(tukey_result)


      Multiple Comparison of Means - Tukey HSD, FWER=0.05      
   group1       group2    meandiff p-adj   lower  upper  reject
---------------------------------------------------------------
hypothesis_1 hypothesis_2    0.136    0.0   0.131 0.1411   True
hypothesis_1 hypothesis_3   0.1334    0.0  0.1283 0.1384   True
hypothesis_2 hypothesis_3  -0.0026 0.4097 -0.0077 0.0024  False
---------------------------------------------------------------


hypothesis_2 vs hypothesis_3 간 차이는 작고 통계적으로 유의하지 않다

평균 차이 = -0.0026 (매우 작음)

신뢰구간: 0을 포함함

p-value = 0.4097 → 귀무가설 기각할 수 없음 → 두 모델의 성능은 동일하다고 봄



모델 간 평균 성능 차이를 통계적으로 검증하기 위해 ANOVA 분석을 수행한 뒤,

개별 모델 쌍 간의 유의미한 차이 여부를 평가하기 위해 Tukey HSD 사후검정을 추가로 시행하였다.

특히, 가설 2와 가설 3은 평균 성능이 근접한 것으로 나타나,

두 모델의 실제 성능이 유사한지를 통계적으로 확인하는 데 이 사후검정이 사용되었다.